In [7]:
import os
import skimage
from skimage import io, transform
import math
import keras
import numpy as np
import tensorflow as tf
from keras import layers
from keras.layers import Input, Add, Dense,Lambda, Activation,Reshape, concatenate,ZeroPadding2D, BatchNormalization,Concatenate,GlobalAveragePooling2D, Dropout,Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img  
import cv2

%matplotlib inline

import keras.backend as K
K.set_learning_phase(1)

In [8]:
def grouped_convolution_block(input1, grouped_channels, cardinality, strides):
    init = input1
    group_list = []
    for c in range(cardinality):
        x = Lambda(lambda z: z[:, :, :, c * grouped_channels:(c + 1) * grouped_channels])(input1)
        x = Conv2D(grouped_channels, (3, 3), padding='same', use_bias=False, strides=(strides, strides), kernel_initializer='he_normal', kernel_regularizer=l2(5e-4))(x)
        group_list.append(x)
    group_merge = concatenate(group_list, axis=-1)
    x = BatchNormalization(axis=-1)(group_merge)
    x = Activation('relu')(x)
    return x

def se_bottleneck_block(input1, filters, cardinality=64, strides=1, r=16):
    init = input1

    grouped_channels = int(filters / cardinality)
    if init._keras_shape[-1] != 2 * filters:
        init = Conv2D(filters * 2, (1, 1), padding='same', strides=(strides, strides),use_bias=False, kernel_initializer='he_normal', kernel_regularizer=l2(5e-4))(init)
        init = BatchNormalization(axis=-1)(init)

    X = Conv2D(filters//2, (1, 1), padding='same', use_bias=False, kernel_initializer='he_normal', kernel_regularizer=l2(5e-4))(input1)
    X = BatchNormalization(axis=-1)(X)
    X = Activation('relu')(X)

    X = grouped_convolution_block(X, grouped_channels, cardinality, strides)

    X = Conv2D(filters * 2, (1, 1), padding='same', use_bias=False, kernel_initializer='he_normal', kernel_regularizer=l2(5e-4))(X)
    X = BatchNormalization(axis=-1)(X)

    X_increase = X
    X = GlobalAveragePooling2D(data_format='channels_last')(X)
    X = Reshape([1,1,filters * 2])(X)
    X = Conv2D(filters = filters * 2 // r, kernel_size = (1, 1))(X)
    X = Activation('relu')(X)
    X = Conv2D(filters = filters * 2, kernel_size = (1, 1))(X)
    X = Activation('sigmoid')(X)
    X = layers.add([keras.layers.multiply([X,X_increase]),init])
    X = Activation('relu')(X)
    return X

def SENet_154(input_shape = (224, 224,3), depth=[3,8,36,3], cardinality=64, width=4, r=16):
    X_input = Input(input_shape)
    X = Conv2D(64, (3, 3), padding='same', use_bias=False, strides=(2, 2), kernel_initializer='he_normal', kernel_regularizer=l2(5e-4))(X_input)
    X = BatchNormalization(axis=-1)(X)
    X = Activation('relu')(X)
    X = Conv2D(64, (3, 3), padding='same', use_bias=False, strides=(1, 1), kernel_initializer='he_normal', kernel_regularizer=l2(5e-4))(X)
    X = BatchNormalization(axis=-1)(X)
    X = Activation('relu')(X)
    X = Conv2D(64, (3, 3), padding='same', use_bias=False, strides=(1, 1), kernel_initializer='he_normal', kernel_regularizer=l2(5e-4))(X)
    X = BatchNormalization(axis=-1)(X)
    X = Activation('relu')(X)                     
    X = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(X)
    
    filters = cardinality * width
    filters_list = []
    for i in range(len(depth)):
        filters_list.append(filters)
        filters *= 2 

    for i in range(depth[0]):
        X = se_bottleneck_block(X, filters_list[0], cardinality, strides=1, r=r)

    depth = depth[1:]  
    filters_list = filters_list[1:]  

    for block_idx, n_i in enumerate(depth):
        for i in range(n_i):
            if i == 0:
                X = se_bottleneck_block(X, filters_list[block_idx], cardinality, strides=2, r=r)
            else:
                X = se_bottleneck_block(X, filters_list[block_idx], cardinality, strides=1, r=r)
    X = GlobalAveragePooling2D()(X)
    X = Dropout(0.2)(X)
    X = Dense(2, use_bias=False, activation='softmax')(X)
    model = Model(inputs = X_input, outputs = X, name='SENet-154')

    return model

In [9]:
model = SENet_154()

In [10]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_71 (Conv2D)              (None, 112, 112, 64) 1728        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, 112, 112, 64) 256         conv2d_71[0][0]                  
__________________________________________________________________________________________________
activation_6 (Activation)       (None, 112, 112, 64) 0           batch_normalization_8[0][0]      
__________________________________________________________________________________________________
conv2d_72 

add_3 (Add)                     (None, 56, 56, 512)  0           multiply_3[0][0]                 
                                                                 activation_18[0][0]              
__________________________________________________________________________________________________
activation_23 (Activation)      (None, 56, 56, 512)  0           add_3[0][0]                      
__________________________________________________________________________________________________
conv2d_280 (Conv2D)             (None, 56, 56, 256)  131072      activation_23[0][0]              
__________________________________________________________________________________________________
batch_normalization_22 (BatchNo (None, 56, 56, 256)  1024        conv2d_280[0][0]                 
__________________________________________________________________________________________________
activation_24 (Activation)      (None, 56, 56, 256)  0           batch_normalization_22[0][0]     
__________

lambda_443 (Lambda)             (None, 28, 28, 0)    0           activation_34[0][0]              
__________________________________________________________________________________________________
lambda_444 (Lambda)             (None, 28, 28, 0)    0           activation_34[0][0]              
__________________________________________________________________________________________________
lambda_445 (Lambda)             (None, 28, 28, 0)    0           activation_34[0][0]              
__________________________________________________________________________________________________
lambda_446 (Lambda)             (None, 28, 28, 0)    0           activation_34[0][0]              
__________________________________________________________________________________________________
lambda_447 (Lambda)             (None, 28, 28, 0)    0           activation_34[0][0]              
__________________________________________________________________________________________________
lambda_448

lambda_602 (Lambda)             (None, 28, 28, 8)    0           activation_49[0][0]              
__________________________________________________________________________________________________
lambda_603 (Lambda)             (None, 28, 28, 8)    0           activation_49[0][0]              
__________________________________________________________________________________________________
lambda_604 (Lambda)             (None, 28, 28, 8)    0           activation_49[0][0]              
__________________________________________________________________________________________________
lambda_605 (Lambda)             (None, 28, 28, 8)    0           activation_49[0][0]              
__________________________________________________________________________________________________
lambda_606 (Lambda)             (None, 28, 28, 8)    0           activation_49[0][0]              
__________________________________________________________________________________________________
lambda_607

__________________________________________________________________________________________________
multiply_11 (Multiply)          (None, 28, 28, 1024) 0           activation_62[0][0]              
                                                                 batch_normalization_45[0][0]     
__________________________________________________________________________________________________
add_11 (Add)                    (None, 28, 28, 1024) 0           multiply_11[0][0]                
                                                                 activation_58[0][0]              
__________________________________________________________________________________________________
activation_63 (Activation)      (None, 28, 28, 1024) 0           add_11[0][0]                     
__________________________________________________________________________________________________
conv2d_825 (Conv2D)             (None, 28, 28, 512)  524288      activation_63[0][0]              
__________

...

__________________________________________________________________________________________________
conv2d_2171 (Conv2D)            (None, 14, 14, 16)   0           lambda_2038[0][0]                
__________________________________________________________________________________________________
conv2d_2172 (Conv2D)            (None, 14, 14, 16)   0           lambda_2039[0][0]                
__________________________________________________________________________________________________
conv2d_2173 (Conv2D)            (None, 14, 14, 16)   0           lambda_2040[0][0]                
__________________________________________________________________________________________________
conv2d_2174 (Conv2D)            (None, 14, 14, 16)   0           lambda_2041[0][0]                
__________________________________________________________________________________________________
conv2d_2175 (Conv2D)            (None, 14, 14, 16)   0           lambda_2042[0][0]                
__________

conv2d_2445 (Conv2D)            (None, 14, 14, 16)   0           lambda_2296[0][0]                
__________________________________________________________________________________________________
conv2d_2446 (Conv2D)            (None, 14, 14, 16)   0           lambda_2297[0][0]                
__________________________________________________________________________________________________
conv2d_2447 (Conv2D)            (None, 14, 14, 16)   0           lambda_2298[0][0]                
__________________________________________________________________________________________________
conv2d_2448 (Conv2D)            (None, 14, 14, 16)   0           lambda_2299[0][0]                
__________________________________________________________________________________________________
conv2d_2449 (Conv2D)            (None, 14, 14, 16)   0           lambda_2300[0][0]                
__________________________________________________________________________________________________
conv2d_245

__________________________________________________________________________________________________
lambda_2580 (Lambda)            (None, 14, 14, 16)   0           activation_204[0][0]             
__________________________________________________________________________________________________
lambda_2581 (Lambda)            (None, 14, 14, 16)   0           activation_204[0][0]             
__________________________________________________________________________________________________
lambda_2582 (Lambda)            (None, 14, 14, 16)   0           activation_204[0][0]             
__________________________________________________________________________________________________
lambda_2583 (Lambda)            (None, 14, 14, 16)   0           activation_204[0][0]             
__________________________________________________________________________________________________
lambda_2584 (Lambda)            (None, 14, 14, 16)   0           activation_204[0][0]             
__________

lambda_2875 (Lambda)            (None, 14, 14, 0)    0           activation_224[0][0]             
__________________________________________________________________________________________________
lambda_2876 (Lambda)            (None, 14, 14, 0)    0           activation_224[0][0]             
__________________________________________________________________________________________________
lambda_2877 (Lambda)            (None, 14, 14, 0)    0           activation_224[0][0]             
__________________________________________________________________________________________________
lambda_2878 (Lambda)            (None, 14, 14, 0)    0           activation_224[0][0]             
__________________________________________________________________________________________________
lambda_2879 (Lambda)            (None, 14, 14, 0)    0           activation_224[0][0]             
__________________________________________________________________________________________________
lambda_288

conv2d_3409 (Conv2D)            (None, 1, 1, 4096)   1052672     activation_251[0][0]             
__________________________________________________________________________________________________
activation_252 (Activation)     (None, 1, 1, 4096)   0           conv2d_3409[0][0]                
__________________________________________________________________________________________________
multiply_49 (Multiply)          (None, 7, 7, 4096)   0           activation_252[0][0]             
                                                                 batch_normalization_161[0][0]    
__________________________________________________________________________________________________
add_49 (Add)                    (None, 7, 7, 4096)   0           multiply_49[0][0]                
                                                                 activation_248[0][0]             
__________________________________________________________________________________________________
activation